<a href="https://colab.research.google.com/github/sambitmishra98/PyFR-ideal-performance/blob/main/performance_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projected performance computation from mesh

In this document, we aim to compare the performance of expected PyFR performance in comparison with actual performance.

The expected performance is computed by understanding inputs and outputs to all kernels used for computation.

The actual performance is computed using `perf_counter()` and looking at wall-time in the solution file. More details are available in https://docs.google.com/spreadsheets/d/1M6Nl5DxMhgi1SuUoGmmD3tR5jjmDrSFV1EWkhQYbmt4/edit?usp=sharing

## Performance details from configuration file

In [ ]:
# [backend]
precision = 'double' # or 'single'

# [solver]
system = 'navier-stokes' # or 'euler'
order = 1

# [solver-time-integrator]
scheme = 'rk4' # or 'rk'